In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import zipfile
import io

In [ ]:
# ✅ Definir corretamente o diretório base dentro do seu workspace
PASTA_BASE = "/home/danielbarbosa/seget/notebooks/dados_abertos_cnpj/arquivos"

# Criar o diretório base se não existir
os.makedirs(PASTA_BASE, exist_ok=True)

# Definir permissões para garantir acesso
os.chmod(PASTA_BASE, 0o755)

# URL principal dos dados
BASE_URL = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/"

In [ ]:
# 1️⃣ BAIXAR A PÁGINA PRINCIPAL E EXTRAIR OS DIRETÓRIOS
print("🔄 Obtendo a lista de diretórios...")
response = requests.get(BASE_URL)
soup = BeautifulSoup(response.text, "html.parser")

# Encontrar todos os links que correspondem a diretórios (exemplo: "2025-02/")
pastas = [a['href'] for a in soup.find_all("a", href=True) if a['href'].endswith("/") and a['href'] != "../"]
print(f"📁 Diretórios encontrados: {pastas}")

In [ ]:
# 2️⃣ ITERAR SOBRE CADA DIRETÓRIO PARA BAIXAR OS ARQUIVOS ZIP
for pasta in pastas:
    url_pasta = BASE_URL + pasta
    response_pasta = requests.get(url_pasta)
    soup_pasta = BeautifulSoup(response_pasta.text, "html.parser")

    # ✅ Criar a estrutura correta dentro de `arquivos/dados/`
    caminho_pasta_local = os.path.join(PASTA_BASE, "dados", pasta.strip('/'))
    print(f'📂 Criando pasta: {caminho_pasta_local}')
    
    os.makedirs(caminho_pasta_local, exist_ok=True)

    # Definir permissões na pasta criada
    os.chmod(caminho_pasta_local, 0o755)

    # 3️⃣ LISTAR ARQUIVOS ZIP DENTRO DO DIRETÓRIO
    arquivos_zip = [a['href'] for a in soup_pasta.find_all("a", href=True) if a['href'].endswith(".zip")]
    print(f"📂 {pasta} - Arquivos ZIP encontrados: {arquivos_zip}")

    for arquivo_zip in arquivos_zip:
        url_arquivo = url_pasta + arquivo_zip
        caminho_arquivo_zip = os.path.join(caminho_pasta_local, arquivo_zip)

        # 4️⃣ FAZER DOWNLOAD DO ARQUIVO ZIP SE ELE NÃO EXISTIR
        if not os.path.exists(caminho_arquivo_zip):
            print(f"⬇️ Baixando {url_arquivo}...")
            response_zip = requests.get(url_arquivo, stream=True)

            if response_zip.status_code == 200:
                with open(caminho_arquivo_zip, "wb") as f:
                    f.write(response_zip.content)
                print(f"✅ Download concluído: {caminho_arquivo_zip}")
            else:
                print(f"❌ Erro ao baixar {url_arquivo}")

        # 5️⃣ EXTRAIR O CONTEÚDO DO ZIP NA MESMA ESTRUTURA
        with zipfile.ZipFile(caminho_arquivo_zip, "r") as zip_ref:
            zip_ref.extractall(caminho_pasta_local)
            print(f"📂 Arquivo extraído em: {caminho_pasta_local}")

print("✅ Processo concluído! Todos os arquivos foram baixados e extraídos.")